In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2022-3"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=ALL_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_exports"] = df["ALL_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_exports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_exports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list

['',
 '1220',
 '2010',
 '5700',
 '5880',
 '4120',
 '4280',
 '5800',
 '4210',
 '3510',
 '5820',
 '4279',
 '4231',
 '5590',
 '5830',
 '5330',
 '6021',
 '4419',
 '5200',
 '4759',
 '3010',
 '5170',
 '3370',
 '5081',
 '5570',
 '4700',
 '5490',
 '4890',
 '4190',
 '3330',
 '3570']

In [8]:
end_use = "hs?get=CTY_NAME,ALL_VAL_MO,E_COMMODITY,E_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\exports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\exports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
4190
<Response [200]>
3330
<Response [200]>
3570
<Response [200]>


In [9]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])

for xxx in country_list:
    
    file = ".\\data\\exports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\exports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
    
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["exports"] = df["ALL_VAL_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"exports": "sum"})
                   
total.reset_index(inplace = True)
                   
total["E_COMMODITY"] = "00"
total["E_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)                   

df.drop(["ALL_VAL_MO", "COMM_LVL", "CTY_CODE"], axis = 1, inplace = True)

df["E_COMMODITY_SDESC"] = "HS CODE " + df["E_COMMODITY"] + ", " + df["E_COMMODITY_SDESC"]

df["E_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","E_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [10]:
df.tail()

E_COMMODITY       exports
CTY_NAME       E_COMMODITY_SDESC time                                
UNITED KINGDOM ALL PRODUCTS      2021-11-01          00  5.307813e+09
                                 2021-12-01          00  5.282541e+09
                                 2022-01-01          00  5.387851e+09
                                 2022-02-01          00  5.486830e+09
                                 2022-03-01          00  6.319719e+09

In [11]:
total[total.CTY_NAME == "TOTAL FOR ALL COUNTRIES"]

,CTY_NAME,time,exports,E_COMMODITY,E_COMMODITY_SDESC
2997,TOTAL FOR ALL COUNTRIES,2013-01-01,1.230319e+11,00,ALL PRODUCTS
2998,TOTAL FOR ALL COUNTRIES,2013-02-01,1.234436e+11,00,ALL PRODUCTS
2999,TOTAL FOR ALL COUNTRIES,2013-03-01,1.366422e+11,00,ALL PRODUCTS
3000,TOTAL FOR ALL COUNTRIES,2013-04-01,1.294606e+11,00,ALL PRODUCTS
3001,TOTAL FOR ALL COUNTRIES,2013-05-01,1.329687e+11,00,ALL PRODUCTS
...,...,...,...,...,...
3103,TOTAL FOR ALL COUNTRIES,2021-11-01,1.563914e+11,00,ALL PRODUCTS
3104,TOTAL FOR ALL COUNTRIES,2021-12-01,1.591364e+11,00,ALL PRODUCTS
3105,TOTAL FOR ALL COUNTRIES,2022-01-01,1.473490e+11,00,ALL PRODUCTS
3106,TOTAL FOR ALL COUNTRIES,2022-02-01,1.506820e+11,00,ALL PRODUCTS
